<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ML-Assignment-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Improting everything
!pip install torch

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Question 1

In [2]:
class XNORNet(nn.Module):
    def __init__(self):
        super(XNORNet, self).__init__()
        # 2 input neurons, 2 hidden neurons, 1 output neuron
        self.layer1 = nn.Linear(2, 2)
        self.layer2 = nn.Linear(2, 1)

    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = torch.sigmoid(self.layer2(x))
        return x

In [3]:
# Training data for XNOR
inputs = torch.tensor([[0.0, 0.0],
                       [0.0, 1.0],
                       [1.0, 0.0],
                       [1.0, 1.0]])
targets = torch.tensor([[1.0],
                        [0.0],
                        [0.0],
                        [1.0]])

# Create the network, define loss function and optimizer
net = XNORNet()
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.5, momentum=0.9)

# Training loop
for epoch in range(10000):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch [{epoch + 1}/10000], Loss: {loss.item():.4f}")

print("Training complete!")

Epoch [1/10000], Loss: 0.2525
Epoch [1001/10000], Loss: 0.0008
Epoch [2001/10000], Loss: 0.0003
Epoch [3001/10000], Loss: 0.0002
Epoch [4001/10000], Loss: 0.0001
Epoch [5001/10000], Loss: 0.0001
Epoch [6001/10000], Loss: 0.0001
Epoch [7001/10000], Loss: 0.0001
Epoch [8001/10000], Loss: 0.0001
Epoch [9001/10000], Loss: 0.0001
Training complete!


In [4]:
with torch.no_grad():
    test = torch.tensor([[0.0, 0.0],
                         [0.0, 1.0],
                         [1.0, 0.0],
                         [1.0, 1.0]])
    predicted = net(test)
    for i, input_tuple in enumerate(test):
        print(f"{input_tuple[0]} XNOR {input_tuple[1]} = {predicted[i][0]:.4f}")

0.0 XNOR 0.0 = 0.9934
0.0 XNOR 1.0 = 0.0068
1.0 XNOR 0.0 = 0.0068
1.0 XNOR 1.0 = 0.9917


# Question 2

In [6]:
class BinaryConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(BinaryConv2d, self).__init__()
        # The real-valued weights
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels // groups, kernel_size, kernel_size))
        self.bias = nn.Parameter(torch.Tensor(out_channels)) if bias else None

        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups

        # Initialize weights and biases
        nn.init.kaiming_uniform_(self.weight, a=5)
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / fan_in ** 0.5
            nn.init.uniform_(self.bias, -bound, bound)

    def binarize(self, x):
        return x.sign()

    def forward(self, x):
        # Binarize the weights
        bin_weight = self.binarize(self.weight)

        # Convolution with binary weights
        return F.conv2d(x, bin_weight, self.bias, self.stride, self.padding, self.dilation, self.groups)

# Test the binary convolution
x = torch.randn(32, 3, 32, 32)  # Batch of 32 images, 3 channels, 32x32 resolution
binary_conv = BinaryConv2d(3, 64, 3, padding=1)  # 3 input channels, 64 output channels, 3x3 kernel
out = binary_conv(x)
print(out.shape)  # [32, 64, 32, 32]

torch.Size([32, 64, 32, 32])
